In [47]:
import numpy as np
import matplotlib.pyplot as plt
import adabmDCA

from adabmDCA.utils import get_device, get_dtype, get_mask_save
from adabmDCA.sampling import get_sampler

import sys
sys.path.append('..')
import selex_distribution, energy_models, tree, data_loading, training, callback, sampling

ImportError: cannot import name 'compute_pearson' from 'utils' (/Users/scrotti/projects/Aptamer2025py/experiments/../utils.py)

In [10]:
import torch

In [11]:
from pathlib import Path
import glob

In [39]:
from importlib import reload
reload(utils)

<module 'utils' from '/Users/scrotti/projects/Aptamer2025py/experiments/../utils.py'>

In [40]:
round_ids = ['Input_R1_N', 'OplusR1_N', 'OplusR2_N']

In [43]:
sequences = []
for round_id in round_ids:
    sequences.append(utils.sequences_from_file_ab6(round_id))

In [45]:
sequences_oh = [utils.one_hot(seq, num_classes=21) for seq in sequences]

In [46]:
tr = tree.Tree()
tr.add_node(-1, name = 'OplusR1_N')
tr.add_node(0, name = 'OplusR2_N')

selected_modes = torch.BoolTensor(
    [[1], [1]]
)

L, q = sequences_oh[0][0].shape

k = torch.zeros(L, q, dtype=dtype)
h = torch.zeros(L, q, dtype=dtype)
J = torch.zeros(L, q, L, q, dtype=dtype)

Ns0 = energy_models.IndepSites(k)
potts = energy_models.Potts(J, h)

ps = selex_distribution.MultiModeDistribution(potts, normalized=False)
model = selex_distribution.MultiRoundDistribution(Ns0, ps, tr, selected_modes)

NameError: name 'tree' is not defined